# The objective here is to split the massive MERSCOPE data into smaller tiles

tiles should overlap 5%

In [1]:
import pandas as pd
import numpy as np

In [2]:
dat = pd.read_csv('../data/detected_transcripts_wPixel.csv')

In [4]:
dat.describe()

,Unnamed: 0,barcode_id,global_x,global_y,global_z,x,y,fov,global_xpixel,global_ypixel
count,4.446952e+07,4.446952e+07,4.446952e+07,4.446952e+07,4.446952e+07,4.446952e+07,4.446952e+07,4.446952e+07,4.446952e+07,4.446952e+07
mean,8.004014e+05,1.783306e+02,6.114530e+03,4.324178e+03,3.100593e+00,1.018254e+03,1.045664e+03,7.274079e+02,5.661598e+04,4.003866e+04
std,5.252250e+05,1.153013e+02,2.221352e+03,1.738564e+03,1.983071e+00,5.299624e+02,5.247331e+02,3.209929e+02,2.056806e+04,1.609781e+04
min,0.000000e+00,0.000000e+00,1.107802e+01,2.008378e+01,0.000000e+00,9.800000e+01,9.800000e+01,0.000000e+00,1.025742e+02,1.859608e+02
25%,3.478990e+05,7.600000e+01,4.488138e+03,2.969954e+03,1.000000e+00,5.631215e+02,6.017426e+02,4.720000e+02,4.155680e+04,2.749956e+04
50%,7.928260e+05,1.880000e+02,6.050912e+03,4.351832e+03,3.000000e+00,1.014617e+03,1.059576e+03,7.380000e+02,5.602693e+04,4.029471e+04
75%,1.160546e+06,2.650000e+02,7.810274e+03,5.788341e+03,5.000000e+00,1.475154e+03,1.494643e+03,1.005000e+03,7.231730e+04,5.359572e+04
max,2.359382e+06,4.610000e+02,1.161126e+04,7.610976e+03,6.000000e+00,1.950000e+03,1.950000e+03,1.293000e+03,1.075116e+05,7.047195e+04


In [5]:
# getting coordinate limits
x_range = dat.global_xpixel.max() - dat.global_xpixel.min()
y_range = dat.global_ypixel.max() - dat.global_ypixel.min()
print(x_range, y_range)

107409.01039032046 70285.98771016506


In [9]:
# maybe split axes into 4...
y_tile_start = np.floor(y_range / 4)
x_tile_start = np.floor(x_range / 4)

In [14]:
y_bump = y_tile_start*0.05
x_bump = x_tile_start*0.05

In [15]:
coords = []
for i in range(5):
    for j in range(5):
        if i == 4 and j == 4:
            x1 = x_tile_start*i - x_bump
            x2 = dat.global_xpixel.max() + x_bump
            y1 = y_tile_start*j - y_bump
            y2 = dat.global_ypixel.max() + y_bump
        elif i == 4:
            x1 = x_tile_start*i - x_bump
            x2 = dat.global_xpixel.max() + x_bump
            y1 = y_tile_start*j - y_bump
            y2 = y_tile_start*(j+1) + y_bump
        elif j == 4:
            x1 = x_tile_start*i - x_bump
            x2 = x_tile_start*(i+1) + x_bump
            y1 = y_tile_start*j - y_bump
            y2 = dat.global_ypixel.max() + y_bump
        else:
            x1 = x_tile_start*i - x_bump
            x2 = x_tile_start*(i+1) + x_bump
            y1 = y_tile_start*j - y_bump
            y2 = y_tile_start*(j+1) + y_bump
        coords.append((x1, x2, y1, y2))
coords

[(-1342.6000000000001, 28194.6, -878.5500000000001, 18449.55),
 (-1342.6000000000001, 28194.6, 16692.45, 36020.55),
 (-1342.6000000000001, 28194.6, 34263.45, 53591.55),
 (-1342.6000000000001, 28194.6, 51834.45, 71162.55),
 (-1342.6000000000001, 28194.6, 69405.45, 71350.49854712789),
 (25509.4, 55046.6, -878.5500000000001, 18449.55),
 (25509.4, 55046.6, 16692.45, 36020.55),
 (25509.4, 55046.6, 34263.45, 53591.55),
 (25509.4, 55046.6, 51834.45, 71162.55),
 (25509.4, 55046.6, 69405.45, 71350.49854712789),
 (52361.4, 81898.6, -878.5500000000001, 18449.55),
 (52361.4, 81898.6, 16692.45, 36020.55),
 (52361.4, 81898.6, 34263.45, 53591.55),
 (52361.4, 81898.6, 51834.45, 71162.55),
 (52361.4, 81898.6, 69405.45, 71350.49854712789),
 (79213.4, 108750.6, -878.5500000000001, 18449.55),
 (79213.4, 108750.6, 16692.45, 36020.55),
 (79213.4, 108750.6, 34263.45, 53591.55),
 (79213.4, 108750.6, 51834.45, 71162.55),
 (79213.4, 108750.6, 69405.45, 71350.49854712789),
 (106065.4, 108854.18456156076, -878.55

In [29]:
for c_i in range(len(coords)):
    c = coords[c_i]
    print(c)
    tile = dat.loc[(dat['global_xpixel'] > c[0]) & (dat['global_xpixel'] < c[1]), :]
    tile = tile.loc[(tile['global_ypixel'] > c[2]) & (tile['global_ypixel'] < c[3]), :]
    if tile.shape[0] > 50:
        print(tile.shape)
        tile.to_csv('../data/tiled_transcripts/tile_{}.csv'.format(c_i))

(-1342.6000000000001, 28194.6, -878.5500000000001, 18449.55)
(51352, 12)
(-1342.6000000000001, 28194.6, 16692.45, 36020.55)
(770558, 12)
(-1342.6000000000001, 28194.6, 34263.45, 53591.55)
(1022170, 12)
(-1342.6000000000001, 28194.6, 51834.45, 71162.55)
(2653334, 12)
(-1342.6000000000001, 28194.6, 69405.45, 71350.49854712789)
(121816, 12)
(25509.4, 55046.6, -878.5500000000001, 18449.55)
(2302025, 12)
(25509.4, 55046.6, 16692.45, 36020.55)
(2956229, 12)
(25509.4, 55046.6, 34263.45, 53591.55)
(6228300, 12)
(25509.4, 55046.6, 51834.45, 71162.55)
(8810600, 12)
(25509.4, 55046.6, 69405.45, 71350.49854712789)
(287828, 12)
(52361.4, 81898.6, -878.5500000000001, 18449.55)
(2721606, 12)
(52361.4, 81898.6, 16692.45, 36020.55)
(7660187, 12)
(52361.4, 81898.6, 34263.45, 53591.55)
(7740502, 12)
(52361.4, 81898.6, 51834.45, 71162.55)
(2853597, 12)
(52361.4, 81898.6, 69405.45, 71350.49854712789)
(79213.4, 108750.6, -878.5500000000001, 18449.55)
(350131, 12)
(79213.4, 108750.6, 16692.45, 36020.55)
(518

In [25]:
tile = dat.loc[(dat['global_xpixel'] > coords[1][0]) & (dat['global_xpixel'] < coords[1][1]), :]

In [26]:
tile = tile.loc[(tile['global_ypixel'] > coords[1][2]) & (tile['global_ypixel'] < coords[1][3]), :]

In [27]:
tile.shape

(770558, 12)